In [ ]:
%pwd

In [ ]:
import os
os.chdir("../")

In [ ]:
%pwd

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
# Extract the data from pdf file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents = loader.load()
    
    return documents


extracted_data = load_pdf_file(data = 'Data/')

In [ ]:
# extracted_data

In [ ]:
# Split the data into text chuncks
def text_split(extracted_data):
        
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

text_chunks = text_split(extracted_data)
print("Length of text chunks", len(text_chunks))

In [8]:
# text_chunks

In [ ]:
# pip install sentence-transformers

In [ ]:
# Embeddings 

from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
def download_hugging_face_embeddings():
    
    embeddings=HuggingFaceEmbeddings()
    return embeddings


embeddings = download_hugging_face_embeddings()
